In [5]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Use the citipy module to determine city based on latitude and longitude.The citipy module finds the nearest city to 
# the latitude and longitude pair with a population of 500 or more.
from citipy import citipy

In [3]:
# Create a set of random latitude and longitude combinations. lats = lattitudes, lngs =  longitudes. zip saves it as a zip
# object which pachs each pair of lats and lngs having the same index in their respective array into a tuple.  If there are
# 1,500 lats and lngs, there will be 1,500 tuples of paired lats and lngs, where each lats and lngs in a tuple can be
# accessed by the index of 0 and 1, respectively. 
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)
lat_lngs

In [4]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [6]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

# Some of this code should look familiar, but let's break it down:

# We create a cities list to store city names.
# We iterate through the coordinates, as in our practice, and retrieve the nearest city using the lats and longs pair.
# We add a decision statement with the logical operator not in to determine whether the found city is already in the cities
# list. If not, then we'll use the append() function to add it. We are doing this because among the 1,500 latitudes and 
# longitudes, there might be duplicates, which will retrieve duplicate cities, and we want to be sure we capture only the 
# unique cities.

613